In [40]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import Series
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools

In [123]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
*The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [11]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import Series
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools
import datetime

In [77]:
def get_all_predictions(dataframe):
    pred_df = pd.DataFrame(index_for_df)
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    pdqs = [(x[0], x[1], x[2], 1) for x in list(itertools.product(p, d, q))]
    df = dataframe.drop('5. volume', axis = 1)
    for metric in df.columns:
        ans = []
        for comb in pdq:
            for combs in pdqs:
                try:
                    mod = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
                    output = mod.fit()
                    ans.append([comb, combs, output.aic])
                except:
                    continue
        ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
        ARIMA_MODEL = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                order=ans_df.loc[ans_df['aic'].idxmin()][0],
                                seasonal_order=ans_df.loc[ans_df['aic'].idxmin()][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
#         print(ans_df.loc[ans_df['aic'].idxmin()][0])
#         print(ans_df.loc[ans_df['aic'].idxmin()][1])
        output = ARIMA_MODEL.fit()
#         pred = output.get_prediction(start=pd.to_datetime('2018-08-31 15:45:00'), end=pd.to_datetime('2018-08-31 14:00:00'),dynamic=False)
#         predictions = pd.DataFrame(pred.predicted_mean)
#         pred_df[metric] = predictions
        forecast = output.get_forecast(steps=10)
        forecast_values = pd.DataFrame(forecast.predicted_mean)
        pred_df[metric] = forecast_values
        print(forecast_values)
    return pred_df

In [8]:
# index_for_df = apple.index[(apple.index < '2018-09-04 15:50:00')& (apple.index > '2018-09-04 10:00:00')]

In [75]:
pred_df = pd.DataFrame(index_for_df)

In [9]:
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
stock = TimeSeries(key='1250F9WWA3Z77BIK')

In [66]:
index_for_df = [pd.to_datetime('2018-09-04 11:05:00')+ datetime.timedelta(minutes=5*x) for x in range(0, 72)]

In [3]:
nflx = pd.read_csv('./csv/nflx_daily_stock.csv', index_col='Unnamed: 0')
fb = pd.read_csv('./csv/fb_daily_stock.csv', index_col='Unnamed: 0')
apple = pd.read_csv('./csv/apple_daily_stock.csv', index_col='Unnamed: 0')
fb_prediction_df = pd.read_csv('./csv/fb_predictions.csv', index_col='Unnamed: 0')

In [68]:
fb_sept, meta_sept = stock.get_intraday(symbol ='FB', interval='5min')

In [69]:
fb_today = pd.DataFrame(fb_sept)
fb_today = pd.DataFrame(fb_today.T)
fb_today.index = pd.to_datetime(fb_today.index)
fb_today['1. open'] = fb_today['1. open'].astype(float)
fb_today['2. high'] = fb_today['2. high'].astype(float)
fb_today['3. low'] = fb_today['3. low'].astype(float)
fb_today['4. close'] = fb_today['4. close'].astype(float)
fb_today['5. volume'] = fb_today['5. volume'].astype(float)

In [80]:
fb_today_graph = fb_today[(fb_today.index > '2018-09-04')]
fb_today = fb_today[(fb_today.index > '2018-09-04')]

In [81]:
get_all_predictions(fb_today)

                              0
2018-09-04 09:25:00  175.267739
2018-09-04 09:20:00  176.969643
2018-09-04 09:15:00  178.671546
2018-09-04 09:10:00  180.373449
2018-09-04 09:05:00  182.075352
2018-09-04 09:00:00  183.777255
2018-09-04 08:55:00  185.479158
2018-09-04 08:50:00  187.181061
2018-09-04 08:45:00  188.882964
2018-09-04 08:40:00  190.584867
                           0
2018-09-04 09:25:00  175.395
2018-09-04 09:20:00  176.970
2018-09-04 09:15:00  178.545
2018-09-04 09:10:00  180.120
2018-09-04 09:05:00  181.695
2018-09-04 09:00:00  183.270
2018-09-04 08:55:00  184.845
2018-09-04 08:50:00  186.420
2018-09-04 08:45:00  187.995
2018-09-04 08:40:00  189.570
                         0
2018-09-04 09:25:00  171.8
2018-09-04 09:20:00  171.8
2018-09-04 09:15:00  171.8
2018-09-04 09:10:00  171.8
2018-09-04 09:05:00  171.8
2018-09-04 09:00:00  171.8
2018-09-04 08:55:00  171.8
2018-09-04 08:50:00  171.8
2018-09-04 08:45:00  171.8
2018-09-04 08:40:00  171.8
                              0


,0,1. open,2. high,3. low,4. close
0,2018-09-04 11:05:00,NaN,NaN,NaN,NaN
1,2018-09-04 11:10:00,NaN,NaN,NaN,NaN
2,2018-09-04 11:15:00,NaN,NaN,NaN,NaN
3,2018-09-04 11:20:00,NaN,NaN,NaN,NaN
4,2018-09-04 11:25:00,NaN,NaN,NaN,NaN
5,2018-09-04 11:30:00,NaN,NaN,NaN,NaN
6,2018-09-04 11:35:00,NaN,NaN,NaN,NaN
7,2018-09-04 11:40:00,NaN,NaN,NaN,NaN
8,2018-09-04 11:45:00,NaN,NaN,NaN,NaN
9,2018-09-04 11:50:00,NaN,NaN,NaN,NaN


# Candlestick Charts 8-31-2018

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

# ts_close = pd.DataFrame(fb['4. close'])
# ts_close = ts_close['4. close'].astype(float)
# rolmean = ts_close.rolling(window=6).mean()
# orig_close = plt.plot(ts, color='blue',label='Original')
# mean_close = plt.plot(rolmean, color='red', label='Rolling Mean')
#     std = plt.plot(rolstd, color='black', label = 'Rolling Std')
# plt.legend(loc='best')
# plt.title('Rolling Mean & Standard Deviation')
# plt.show(block=False)


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Facebook Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Price',x=nflx.index,
                       open=fb_today_graph['1. open'],
                       high=fb_today_graph['2. high'],
                       low=fb_today_graph['3. low'],
                       close=fb_today_graph['4. close'],
                       increasing=dict(line=dict(color= 'Blue')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=fb_prediction_df.index,
                       open=fb_prediction_df.open_pred,
                       high=fb_prediction_df.high_pred,
                       low=fb_prediction_df.low_pred,
                       close=fb_prediction_df.close_pred,
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))

# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (15)')

In [130]:
join_fb = fb[(fb.index < '2018-08-31 15:50:00') & (fb.index > '2018-08-31 13:55:00')]

In [131]:
join_fb.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
fb_join = fb_prediction_df.join(join_fb)
fb_join['close_difference'] = fb_join.close_pred - fb_join.actual_close
fb_join['open_difference'] = fb_join.open_pred - fb_join.actual_open
fb_join['high_difference'] = fb_join.high_pred - fb_join.actual_high
fb_join['low_difference'] = fb_join.low_pred - fb_join.actual_low

In [137]:
display(fb_join.close_difference.describe())
display(fb_join.open_difference.describe())
display(fb_join.high_difference.describe())
display(fb_join.low_difference.describe())

count    22.000000
mean      0.006818
std       0.138313
min      -0.235100
25%      -0.083675
50%       0.010000
75%       0.067500
max       0.340000
Name: close_difference, dtype: float64

count    22.000000
mean     -0.004545
std       0.131522
min      -0.220000
25%      -0.110000
50%       0.015000
75%       0.047425
max       0.300100
Name: open_difference, dtype: float64

count    22.000000
mean      0.012018
std       0.137539
min      -0.286967
25%      -0.051162
50%       0.009452
75%       0.082324
max       0.335000
Name: high_difference, dtype: float64

count    22.000000
mean      0.002415
std       0.131138
min      -0.249679
25%      -0.060606
50%      -0.004802
75%       0.081127
max       0.325198
Name: low_difference, dtype: float64

In [93]:
apple_prediction_df = get_all_predictions(apple)

In [98]:
apple_prediction_df.to_csv('apple_predictions.csv')

In [97]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Apple Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Actual Price',x=apple.index,
                       open=apple['1. open'],
                       high=apple['2. high'],
                       low=apple['3. low'],
                       close=apple['4. close'],
                       increasing=dict(line=dict(color= 'Black')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=apple_prediction_df.index,
                       open=apple_prediction_df['1. open'],
                       high=apple_prediction_df['2. high'],
                       low=apple_prediction_df['3. low'],
                       close=apple_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))


# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace,trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (16)')

In [138]:
join_apple = apple[(apple.index < '2018-08-31 15:50:00') & (apple.index > '2018-08-31 13:55:00')]

In [143]:
join_apple.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
apple_join = apple_prediction_df.join(join_apple)
apple_join['close_difference'] = apple_join['4. close'] - apple_join.actual_close
apple_join['open_difference'] = apple_join['1. open'] - apple_join.actual_open
apple_join['high_difference'] = apple_join['2. high'] - apple_join.actual_high
apple_join['low_difference'] = apple_join['3. low'] - apple_join.actual_low

In [144]:
display(apple_join.close_difference.describe())
display(apple_join.open_difference.describe())
display(apple_join.high_difference.describe())
display(apple_join.low_difference.describe())

count    22.000000
mean      0.013465
std       0.194486
min      -0.275567
25%      -0.116620
50%      -0.031214
75%       0.068373
max       0.475825
Name: close_difference, dtype: float64

count    22.000000
mean      0.002605
std       0.211075
min      -0.326030
25%      -0.129948
50%      -0.029738
75%       0.072788
max       0.492722
Name: open_difference, dtype: float64

count    22.000000
mean      0.004091
std       0.158138
min      -0.300100
25%      -0.092425
50%       0.004950
75%       0.095075
max       0.300000
Name: high_difference, dtype: float64

count    22.000000
mean     -0.004414
std       0.167489
min      -0.335100
25%      -0.136276
50%       0.004879
75%       0.110213
max       0.260603
Name: low_difference, dtype: float64

In [118]:
netflix_prediction_df = get_all_predictions(nflx)

In [121]:
netflix_prediction_df.to_csv('netflix_predictions.csv')

In [120]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Netflix Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}

trace = go.Candlestick(name = 'Actual Price',x=nflx.index,
                       open=nflx['1. open'],
                       high=nflx['2. high'],
                       low=nflx['3. low'],
                       close=nflx['4. close'],
                       increasing=dict(line=dict(color= 'Red')),
                       decreasing=dict(line=dict(color= 'Black')))

trace2 = go.Candlestick(name = 'Predicted Price',x=netflix_prediction_df.index,
                       open=netflix_prediction_df['1. open'],
                       high=netflix_prediction_df['2. high'],
                       low=netflix_prediction_df['3. low'],
                       close=netflix_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Silver')))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (18)')

In [146]:
nflx

,1. open,2. high,3. low,4. close,5. volume,interval
2018-08-31 15:55:00,367.6700,367.8000,367.4707,367.8000,992649,5
2018-08-31 15:50:00,367.2300,368.6950,367.2300,367.6541,267010,5
2018-08-31 15:45:00,367.9600,368.0500,367.1200,367.2000,144391,5
2018-08-31 15:40:00,367.5400,367.8420,367.2001,367.8000,77278,5
2018-08-31 15:35:00,367.5699,367.7368,367.4626,367.6400,105822,5
2018-08-31 15:30:00,367.2100,367.7500,367.1100,367.4750,142012,5
2018-08-31 15:25:00,368.0200,368.0500,367.1900,367.2600,150380,5
2018-08-31 15:20:00,369.5200,369.5200,368.1332,368.1400,135714,5
2018-08-31 15:15:00,369.0900,369.4700,369.0000,369.4700,44391,5
2018-08-31 15:10:00,369.4800,369.4800,369.1500,369.1500,32156,5


In [153]:
# nflx = nflx.drop('interval',axis =1)
join_nflx = nflx[(nflx.index < '2018-08-31 15:50:00') & (nflx.index > '2018-08-31 13:55:00')]
join_nflx.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
nflx_join = netflix_prediction_df.join(join_nflx)
nflx_join['close_difference'] = nflx_join['4. close'] - nflx_join.actual_close
nflx_join['open_difference'] = nflx_join['1. open'] - nflx_join.actual_open
nflx_join['high_difference'] = nflx_join['2. high'] - nflx_join.actual_high
nflx_join['low_difference'] = nflx_join['3. low'] - nflx_join.actual_low

In [155]:
display(nflx_join.close_difference.describe())
display(nflx_join.open_difference.describe())
display(nflx_join.high_difference.describe())
display(nflx_join.low_difference.describe())

count    22.000000
mean     -0.217631
std       0.459140
min      -1.576706
25%      -0.434183
50%      -0.173279
75%       0.015096
max       0.454100
Name: close_difference, dtype: float64

count    22.000000
mean     -0.193638
std       0.478764
min      -1.531282
25%      -0.390712
50%      -0.121451
75%       0.151308
max       0.447062
Name: open_difference, dtype: float64

count    22.000000
mean     -0.097944
std       0.405771
min      -1.366914
25%      -0.269099
50%      -0.119491
75%       0.173028
max       0.726956
Name: high_difference, dtype: float64

count    22.000000
mean     -0.171364
std       0.357031
min      -0.943200
25%      -0.246875
50%      -0.140000
75%       0.070000
max       0.378000
Name: low_difference, dtype: float64